# MattFlow

A CFD python package for the shallow water equations. [repo]
___
MattFlow simulates the surface of the water after any initial conditions, such as drops or stones falling on.

All the required steps, being represented at this notebook, are gathered at ```__main__.py```, and the  
simulation can be configured at the _config_ module.  

## How to install and run Mattflow

1. anaconda environment (recomended)

```bash
$ mkdir mattflow
$ cd mattflow
$ conda create --name mattflows python=3
$ conda activate mattflow
$ pip install mattflow
$ mattflow
```

2. venv (python>=3.3)  

```bash
$ mkdir mattflow
$ cd mattflow
$ python3 -m venv mattflow_env
$ source mattflow_env/bin/activate
$ pip install mattflow
$ mattflow
```

3. pip

```bash
$ mkdir mattflow
$ cd mattflow
$ pip install --user mattflow
$ mattflow
```
So, this is an overview of how it works:  

[repo]: <https://github.com/ThanasisMattas/mattflow>

In [1]:
from mattflow import config as conf
from mattflow import logger
from mattflow import initializer
from mattflow import boundaryConditionsManager
from mattflow import mattflow_solver
from mattflow import mattflow_post
import numpy as np

In [2]:
# Pre-processing {
#
# Spatial discretization steps (structured/cartetian mesh)
dx = (conf.MAX_X - conf.MIN_X) / conf.Nx
dy = (conf.MAX_Y - conf.MIN_Y) / conf.Ny

# Cell centers on x and y dimentions
cx = np.arange(conf.MIN_X + (0.5 - conf.Ng) * dx,
                             conf.MAX_X + conf.Ng * dx, dx)
cy = np.arange(conf.MIN_Y + (0.5 - conf.Ng) * dy,
                             conf.MAX_Y + conf.Ng * dy, dy)
#
# }

In [3]:
# Solution {
#
time = 0

In [4]:
# Initialization
U = initializer.initialize(cx, cy)
drops = 1

In [5]:
# This will hold the step-wise solutions for the post-processing animation.
U_stepwise_for_animation = [U[0, conf.Ng: -conf.Ng, conf.Ng: -conf.Ng]]

In [6]:
# This will hold the step-wise time for the post-processing animation.
# time * 10 is appended, because space is scaled about x10
time_array_for_ani = np.array([0])
update_time_array = lambda t : np.hstack((time_array_for_ani, [t * 10]))

In [7]:
for iter in range(1, conf.MAX_ITERS):
    
    # Time discretization step (CFL condition)
    delta_t = mattflow_solver.dt(U, dx, dy)

    # Update current time
    time += delta_t
    if time > conf.STOPPING_TIME:
        break
    time_array_for_ani = update_time_array(time)

    # Apply boundary conditions (reflective)
    U = boundaryConditionsManager.updateGhostCells(U)

    # Numerical iterative scheme - 2-stage Runge-Kutta
    # Total flux entering and leaving a cell is calculated with the
    # Lax-Friedrichs scheme
    U = mattflow_solver.solve(U, dx, cx, dy, cy, delta_t, iter, drops)

    # Append current frame to the list, to be animated at post-processing
    U_stepwise_for_animation = np.append(U_stepwise_for_animation,
        [U[0, conf.Ng: -conf.Ng, conf.Ng: -conf.Ng]], 0)
#
# }

In [8]:
%matplotlib notebook

In [9]:
%%capture

# Post-processing {
#
ani = mattflow_post.createAnimation(U_stepwise_for_animation, cx, cy,
                                    time_array_for_ani)

In [10]:
# play the animation
import matplotlib.pyplot as plt
plt.rcParams['animation.html'] = 'html5'
ani
#
# }

>(C) 2019, Thanasis Mattas  
>atmattas@physics.auth.gr